## Stats for Taylor Diagrams
## pCO2 for corrected models

In [1]:
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import random
import numpy as np
import xarray as xr
import pandas as pd
import joblib
import pickle
import xesmf as xe
import glob

import seaborn as sns
import cmocean as cm            # really nice colorbars
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats

In [2]:
start_yr = 1985
end_yr = 2020
# =========================================
# For accessing directories
# =========================================
root_dir = "/data/artemis/workspace/vbennington/LDEO_HPD/models/XGB/GCB_2021"  # directory output will be written to  
recon_output_dir = f"{root_dir}/reconstructions" # reconstructions saved here

In [3]:
models = [ 'cesm_sfco2_1x1_A','fesom2_sfco2_1x1_A','mpi_sfco2_1x1_A','cnrm_sfco2_1x1_A','ipsl_sfco2_1x1_A',
          'planktom_sfco2_1x1_A','noresm_sfco2_1x1_A','princeton_sfco2_1x1_A']
correcs = ['1998-2020','2000-2020','1982-2020']

In [4]:
# Set up dataframe for climatology-corrected #
################################################################
df_1998_2020 = pd.DataFrame() # create empty data frame
df_1998_2020['stats']= ["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT",
              "R_SOCCOM","STD_SOCCOM","RMSE_SOCCOM", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]

df_1982_2020 = pd.DataFrame() # create empty data frame
df_1982_2020['stats']= ["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT",
              "R_SOCCOM","STD_SOCCOM","RMSE_SOCCOM", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]

df_2000_2020 = pd.DataFrame() # create empty data frame
df_2000_2020['stats']= ["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT",
              "R_SOCCOM","STD_SOCCOM","RMSE_SOCCOM", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]

df_ens = pd.DataFrame() # create empty data frame
df_ens['stats']= ["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT",
              "R_SOCCOM","STD_SOCCOM","RMSE_SOCCOM", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]

df_models = pd.DataFrame() # create empty data frame
df_models['stats']= ["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT",
              "R_SOCCOM","STD_SOCCOM","RMSE_SOCCOM", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]

In [5]:
cdf = ['1998_2020','2000_2020','1982_2020']

In [12]:
# We have 3 different climatology-corrected files (using climatology of different periods):
for correction in correcs:
    print(correction)
    # Load netcdf
    ds = xr.load_dataset(f"{recon_output_dir}/pCO2_cc{correction}_1x1_recon_1959-2020.nc")
    for mod in models:
        print(mod)
        R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(ds,mod,'pCO2cc',start_yr=start_yr,end_yr=end_yr)
        R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(ds,mod,'pCO2cc',start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines
        R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(ds,mod,'pCO2cc',start_yr=start_yr,end_yr=end_yr)
        R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(ds,mod,'pCO2cc',start_yr=start_yr,end_yr=end_yr)
        R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(ds,mod,'pCO2cc',start_yr=start_yr,end_yr=end_yr)

        # Write to DataFrame:
        if correction == "1998-2020":
            df_1998_2020[f'{mod}']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, 
                     R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
        elif correction == "2000-2020":
            df_2000_2020[f'{mod}']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, 
                     R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
        elif correction == "1982-2020":
            df_1982_2020[f'{mod}']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, 
                     R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df_1998_2020.head()

1998-2020
cesm_sfco2_1x1_A


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Create weight file: bilinear_180x360_180x360.nc
Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
fesom2_sfco2_1x1_A


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
mpi_sfco2_1x1_A


...

/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
princeton_sfco2_1x1_A


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.


,stats,cesm_sfco2_1x1_A,fesom2_sfco2_1x1_A,mpi_sfco2_1x1_A,cnrm_sfco2_1x1_A,ipsl_sfco2_1x1_A,planktom_sfco2_1x1_A,noresm_sfco2_1x1_A,princeton_sfco2_1x1_A
0,R_BATS,0.943319,0.939783,0.926579,0.932002,0.943402,0.941251,0.942755,0.930877
1,STD_BATS,31.317127,31.708062,33.364849,31.203496,30.975092,31.756264,30.381791,32.505086
2,RMSE_BATS,12.062760,12.385086,13.628133,13.080910,12.094552,12.160932,12.296391,13.149251
3,R_HOT,0.922580,0.917805,0.889872,0.907450,0.923572,0.918147,0.920879,0.926893
4,STD_HOT,18.453731,19.295748,23.986608,18.388379,18.444599,19.187600,22.369665,18.262956


In [13]:
df_1998_2020.head(20)

,stats,cesm_sfco2_1x1_A,fesom2_sfco2_1x1_A,mpi_sfco2_1x1_A,cnrm_sfco2_1x1_A,ipsl_sfco2_1x1_A,planktom_sfco2_1x1_A,noresm_sfco2_1x1_A,princeton_sfco2_1x1_A
0,R_BATS,0.943319,0.939783,0.926579,0.932002,0.943402,0.941251,0.942755,0.930877
1,STD_BATS,31.317127,31.708062,33.364849,31.203496,30.975092,31.756264,30.381791,32.505086
2,RMSE_BATS,12.062760,12.385086,13.628133,13.080910,12.094552,12.160932,12.296391,13.149251
3,R_HOT,0.922580,0.917805,0.889872,0.907450,0.923572,0.918147,0.920879,0.926893
4,STD_HOT,18.453731,19.295748,23.986608,18.388379,18.444599,19.187600,22.369665,18.262956
5,RMSE_HOT,8.459858,8.408274,11.297976,8.606548,8.212656,9.109678,9.153042,7.425889
6,R_SOCCOM,0.508045,0.467740,0.402506,0.518316,0.494008,0.451347,0.483273,0.532900
7,STD_SOCCOM,19.563756,20.038467,25.177510,18.462427,19.077136,18.675550,19.536488,20.772598
8,RMSE_SOCCOM,32.021291,32.975270,35.635763,31.691589,32.362804,33.266459,32.428392,31.545895
9,R_LDEO,0.834071,0.824081,0.789125,0.826774,0.834223,0.803776,0.818572,0.833082


In [14]:
# Write out Files #
df_1998_2020.to_csv(f'{recon_output_dir}/Taylor_stats_pCO2cc_allmodels_1998-2020_{start_yr}-{end_yr}.csv',index=False)
df_1982_2020.to_csv(f'{recon_output_dir}/Taylor_stats_pCO2cc_allmodels_1982-2020_{start_yr}-{end_yr}.csv',index=False)
df_2000_2020.to_csv(f'{recon_output_dir}/Taylor_stats_pCO2cc_allmodels_2000-2020_{start_yr}-{end_yr}.csv',index=False)

In [15]:
# Now, do for raw models:
# Get model output from prior to 1982:
# Now we need the GCB models pCO2:
dm = xr.open_dataset('/data/artemis/simulations/GCB/2021models/gcb_load_A_2021_models.nc',decode_times=False)
dm = dm.rename({'X':'lon','Y':'lat','T':'time'})
dm['lon'] = list(map(lambda x: x-360 if x>180 else x, dm['lon'].values))
dm = dm.sortby('lon')
dm['time']=pd.date_range(start=f'1958-01T00:00:00.000000000',end=f'2020-12T00:00:00.000000000',freq='MS') + np.timedelta64(14, 'D')
# IPSL is corrected:
#di = xr.open_dataset('/data/artemis/simulations/GCB/2020models/IPSL_corrected.nc',decode_times=False)
#di = di.rename({'months':'time'})
#di['lon'] = list(map(lambda x: x-360 if x>180 else x, di['lon'].values))
#di = di.sortby('lon')
#di['time']=pd.date_range(start=f'1958-01T00:00:00.000000000',end=f'2019-12T00:00:00.000000000',freq='MS') + np.timedelta64(14, 'D')
#dm['ipsl_spco2_1x1_A'] = di['IPSL_pco2_corrected'].transpose("time","lat",'lon')
#del di

In [16]:
print(dm)

<xarray.Dataset>
Dimensions:                (lat: 180, lon: 360, time: 756)
Coordinates:
  * lon                    (lon) float64 -179.5 -178.5 -177.5 ... 178.5 179.5
  * lat                    (lat) float64 -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * time                   (time) datetime64[ns] 1958-01-15 ... 2020-12-15
Data variables:
    area                   (lat, lon) float64 ...
    cesm_fgco2_1x1_A       (time, lat, lon) float64 ...
    cesm_sfco2_1x1_A       (time, lat, lon) float64 ...
    fesom2_fgco2_1x1_A     (time, lat, lon) float64 ...
    fesom2_sfco2_1x1_A     (time, lat, lon) float64 ...
    cnrm_fgco2_1x1_A       (time, lat, lon) float64 ...
    cnrm_sfco2_1x1_A       (time, lat, lon) float64 ...
    ipsl_fgco2_1x1_A       (time, lat, lon) float64 ...
    ipsl_sfco2_1x1_A       (time, lat, lon) float64 ...
    planktom_fgco2_1x1_A   (time, lat, lon) float64 ...
    planktom_sfco2_1x1_A   (time, lat, lon) float64 ...
    noresm_fgco2_1x1_A     (time, lat, lon) float64 ..

In [17]:
# We have 8 models in dm (uncorrected model pCO2)
##################################################
for mod in models:
    print(mod)
    # Load netcdf
    R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(dm,mod,mod,start_yr=start_yr,end_yr=end_yr)
    R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(dm,mod,mod,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines
    R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(dm,mod,mod,start_yr=start_yr,end_yr=end_yr)
    R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(dm,mod,mod,start_yr=start_yr,end_yr=end_yr)
    R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(dm,mod,mod,start_yr=start_yr,end_yr=end_yr)

    df_models[f'{mod}']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, 
                     R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df_models.head(15)

cesm_sfco2_1x1_A


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/xesmf/smm.py:73: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not C_CONTIGUOUS. "


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.


...

Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/xesmf/smm.py:73: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not C_CONTIGUOUS. "


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/xesmf/smm.py:73: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn("Input array is not C_CONTIGUOUS. "


,stats,cesm_sfco2_1x1_A,fesom2_sfco2_1x1_A,mpi_sfco2_1x1_A,cnrm_sfco2_1x1_A,ipsl_sfco2_1x1_A,planktom_sfco2_1x1_A,noresm_sfco2_1x1_A,princeton_sfco2_1x1_A
0,R_BATS,0.929565,0.939886,0.923171,0.908742,0.935126,0.937070,0.862586,0.933324
1,STD_BATS,39.952802,40.709577,46.125806,35.835002,35.386238,36.868450,28.139510,41.297353
2,RMSE_BATS,15.201039,15.931462,24.961138,19.873786,17.263100,13.435975,18.611279,15.160893
3,R_HOT,0.925989,0.924131,0.896702,0.886442,0.919742,0.919850,0.916067,0.924461
4,STD_HOT,20.082740,20.056567,24.928569,19.256582,20.496880,18.601017,22.451326,20.397445
5,RMSE_HOT,7.695605,7.841069,20.864546,15.733327,8.530190,19.713430,28.132643,12.930235
6,R_SOCCOM,0.397877,0.169151,0.231357,0.150604,0.220471,-0.057357,0.171748,0.377914
7,STD_SOCCOM,21.786211,28.366417,35.875386,19.762817,21.258006,21.440540,22.512306,27.017274
8,RMSE_SOCCOM,35.702975,43.697170,45.274858,39.385943,38.429844,44.112806,40.077380,38.434020
9,R_LDEO,0.611889,0.520822,0.533134,0.564289,0.598805,0.480677,0.548732,0.637786


In [18]:
# Write out Files #
df_models.to_csv(f'{recon_output_dir}/Taylor_stats_pCO2_allmodels_{start_yr}-{end_yr}.csv',index=False)

In [19]:
correction = "2000-2020"

In [20]:
# We have 1 final product:
# Doesn't matter which file we open, as correction in all files the same after 1981
print(correction)
# Load netcdf
ds = xr.load_dataset(f"{recon_output_dir}/pCO2_cc{correction}_1x1_recon_1959-2020.nc")
ds['ens'] = ds.pCO2.mean("model") # use HPD, not climatology correction for entire period
R_BATS, STD_BATS, RMSE_BATS, b_obs = BATS_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr)
R_HOT, STD_HOT, RMSE_HOT, h_obs = HOT_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines
R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, s_obs = SOCCOM_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr)
R_LDEO, STD_LDEO, RMSE_LDEO, l_obs = LDEO_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr)
R_GLODAP, STD_GLODAP, RMSE_GLODAP, g_obs = GLODAP_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr)

df_ens['HPD']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, 
                     R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]

df_ens['observed']=[1,b_obs,0,1,h_obs,0,1,s_obs,0,1,l_obs,0,1,g_obs,0]

df_ens.head(15)

2000-2020


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.


,stats,HPD,observed
0,R_BATS,0.945207,1.000000
1,STD_BATS,30.963623,35.604917
2,RMSE_BATS,11.931436,0.000000
3,R_HOT,0.930037,1.000000
4,STD_HOT,20.007301,19.401837
5,RMSE_HOT,7.866122,0.000000
6,R_SOCCOM,0.556732,1.000000
7,STD_SOCCOM,19.866852,36.740302
8,RMSE_SOCCOM,31.321699,0.000000
9,R_LDEO,0.906959,1.000000


In [21]:
# We have 3 different climatology-corrected files (using climatology of different periods):
for correction in correcs:
    print(correction)
    # Load netcdf
    ds = xr.load_dataset(f"{recon_output_dir}/pCO2_cc{correction}_1x1_recon_1959-2020.nc")
    ds['ens'] = ds.pCO2cc.mean("model")
    R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr)
    R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines
    R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr)
    R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr)
    R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(ds,'ens','ens',start_yr=start_yr,end_yr=end_yr)

    df_ens[f'{correction}']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT,R_SOCCOM,STD_SOCCOM,RMSE_SOCCOM, 
                     R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
     
df_ens.head(15)

1998-2020


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
2000-2020


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
1982-2020


/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/home/vbennington/.conda/envs/dev2/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in greater


Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.
Overwrite existing file: bilinear_180x360_180x360.nc 
 You can set reuse_weights=True to save computing time.


,stats,HPD,observed,1998-2020,2000-2020,1982-2020
0,R_BATS,0.945207,1.000000,0.944610,0.944479,0.945228
1,STD_BATS,30.963623,35.604917,31.408612,31.391670,31.315551
2,RMSE_BATS,11.931436,0.000000,11.939852,11.957880,11.863270
3,R_HOT,0.930037,1.000000,0.931573,0.931292,0.931401
4,STD_HOT,20.007301,19.401837,19.449076,19.455413,19.145376
5,RMSE_HOT,7.866122,0.000000,7.771595,7.836608,7.795457
6,R_SOCCOM,0.556732,1.000000,0.541532,0.541984,0.533917
7,STD_SOCCOM,19.866852,36.740302,17.870341,18.046816,17.566228
8,RMSE_SOCCOM,31.321699,0.000000,31.328013,31.335766,31.670893
9,R_LDEO,0.906959,1.000000,0.845073,0.844472,0.843724


In [22]:
# Write out Files #
df_ens.to_csv(f'{recon_output_dir}/Taylor_stats_pCO2_ens_{start_yr}-{end_yr}.csv',index=False)

In [7]:
def BATS_stats(ml_timeseries,model,recon,start_yr=1984,end_yr=2018,blat=121,blon=115):
    
    # determine appropriate START YEAR # 
    if start_yr < 1984:
        start_yr = 1984
  
    if end_yr > 2018:
        end_yr = 2018
    
    # Grab BATS observations:
    bats1 = xr.open_dataset('/data/artemis/workspace/gloege/data/BATS/BATS.nc')
    bats2 = xr.open_dataset('/data/artemis/observations/BATS/2021_data/bats_spco2_199110-201906.nc')
    bats = xr.concat(objs=[bats1,bats2],dim='time')
    # Remove duplicates
    bats = bats.groupby('time').mean()
    # Put BATS monthly averages:
    bats_monthly = bats.spco2.to_dataframe().resample('M').mean()
    # Extract years of interest:
    bats_monthly = bats_monthly.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    # 31 50'N 64 10'W ###########     
    
    # Model already monthly #
    if ((recon=="pCO2cc") & (model!="ens")):
        pco2_df = ml_timeseries[recon].sel(model=f"{model}")[:,blat,blon].squeeze().to_dataframe()
    if model=="ens":
        pco2_df = ml_timeseries[recon][:,blat,blon].squeeze().to_dataframe()
    if ((recon!="pCO2cc") & (model!="ens")):
        pco2_df = ml_timeseries[recon][:,blat,blon].squeeze().to_dataframe()
        
    pco2_df = pco2_df.loc[f'{start_yr}-01-01 00:00:00':f'{end_yr}-12-15 00:00:00']    
    
    # get time uniform:
    bats_monthly = bats_monthly.set_index(pco2_df.index)
        
    # correlate where no NANs
    fib = (~np.isnan(bats_monthly.spco2) & ~np.isnan(pco2_df[recon]))
    R = np.corrcoef(pco2_df[recon][fib],bats_monthly['spco2'][fib])[0,1]
    STD = np.nanstd(pco2_df[recon][fib])
    BATS_STD = np.nanstd(bats_monthly.spco2[fib])
    
    #RMSE
    RMSE = np.sqrt(np.square(bats_monthly.spco2[fib]-pco2_df[recon][fib]).sum()/(fib.sum()))
    
    return R, STD, RMSE, BATS_STD

In [8]:
def HOT_stats(ml_timeseries,model,recon,start_yr=1989,end_yr=2016,hlat=112,hlon=22):
    
    if start_yr < 1989:
        start_yr = 1989
    if end_yr < start_yr:
        print("Your dates are wrong for HOT; data starts in 1989")
    if end_yr > 2016:
        end_yr = 2016
    
    # Grab observations
    hot = xr.open_dataset('/data/artemis/workspace/gloege/data/HOT/HOT.nc')
    hot=hot.where(hot.spco2>0)
    
    if ((recon=="pCO2cc") & (model!='ens')):
        pco2_df = ml_timeseries[recon].sel(model=f"{model}")[:,hlat,hlon].squeeze().to_dataframe()
    if model=="ens":
        pco2_df = ml_timeseries[recon][:,hlat,hlon].squeeze().to_dataframe()
    if ((recon!="pCO2cc") & (model!="ens")):
        pco2_df = ml_timeseries[recon][:,hlat,hlon].squeeze().to_dataframe()
    
    pco2_df = pco2_df.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-15 00:00:00']
    
    
    # P
    hots_monthly = hot.spco2.to_dataframe().resample('M').mean()
    hots_monthly = hots_monthly.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    hots_monthly = hots_monthly.set_index(pco2_df.index)
    # Point by point comparison #
    ind = ((~np.isnan(hots_monthly.spco2)) & (hots_monthly.spco2 > 150) & (~np.isnan(pco2_df[recon])))
    R = np.corrcoef(pco2_df[recon][ind],hots_monthly.spco2[ind])[0,1]
    
    STD = np.std(pco2_df[recon][ind])
    HOT_STD = np.nanstd(hots_monthly.spco2[ind])
    
    #RMSE
    RMSE = np.sqrt(np.square(hots_monthly.spco2[ind]-pco2_df[recon][ind]).sum()/ind.sum())
    
    # We want to know the correlation coef, STD of reconstruction, trendline, seasonal cycle:
    return R, STD, RMSE, HOT_STD

In [9]:
def SOCCOM_stats(ml_gridded,model,recon,start_yr=1982,end_yr=2019):
    
    start_yr = max(1985,start_yr)
    if recon in ['csirml6','jenamls','lsceffnn2','mpisomffn','spco2']:
        end_yr = 2018
        
    # grab observations and years of interest #
    df = xr.open_dataset('/data/artemis/observations/SOCCOM/processed/SOCCOM_gridded_spco2_mon_195001-202112.nc') 
    spco2 = df.spco2[(start_yr-1950)*12:(end_yr-1949)*12,:,:]
    time = df.time[(start_yr-1950)*12:(end_yr-1949)*12]
    lat = df.ylat
    lon = df.xlon
   
    if ((recon=="pCO2cc") & (model!='ens')):
        pco2 = ml_gridded[recon].sel(model=f"{model}").loc[f'{start_yr}-1-01 00:00:00':f'{end_yr+1}-1-01']
    if model=="ens":
        pco2 = ml_gridded[recon].loc[f'{start_yr}-1-01 00:00:00':f'{end_yr+1}-1-01']
    if ((recon!="pCO2cc") & (model!="ens")):
        pco2 = ml_gridded[recon].loc[f'{start_yr}-1-01 00:00:00':f'{end_yr+1}-1-01']
        
    # Grab from model gridded reconstruction #
    ylat = ml_gridded.lat
    xlon = ml_gridded.lon
    pco2_stack = pco2.stack(level=['time','lat','lon'])
    
    # stack the time series #
    soccom = spco2.stack(level=['time','ylat','xlon'])
    
    fis = ((~np.isnan(soccom.values)) & (~np.isnan(pco2_stack.values)) & (soccom.values < 815) & (soccom.values > 150))
    
    R = np.corrcoef(soccom[fis],pco2_stack[fis])[0,1]
    STD = np.std(pco2_stack[fis].values)
    SOCCOM_STD = np.std(soccom[fis].values)
    
    #RMSE
    RMSE = np.sqrt(np.square(soccom[fis].values-pco2_stack[fis].values).sum()/fis.sum())
    
    return R, STD, RMSE, SOCCOM_STD

In [10]:
def LDEO_stats(ml_gridded,model,recon,start_yr=1982,end_yr=2018):
    
    start_yr = max(start_yr,1985)
    if end_yr > 2018:
        end_yr = 2018

    lon = ml_gridded.lon
    lat = ml_gridded.lat
    mtime = ml_gridded.time

    # Load the observations #
    ldeo = xr.open_dataset('/data/artemis/observations/LDEO_database/processed/LDEOv2018_1x1_198201-201812.nc')
    ldeo_pco2=ldeo.spco2_mean[(start_yr-1982)*12:(end_yr-1981)*12,:,:]
    ldeo_lat=ldeo.lat
    ldeo_lon=ldeo.lon
    ldeo_time = ldeo.time
    
    # Regrid to LDEO's grid #########
    mgrid = xr.Dataset({'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    lgrid = xr.Dataset({'lat':(['lat'],ldeo_lat.values),'lon':(['lon'],ldeo_lon.values)})
    if ((recon=="pCO2cc") & (model!='ens')):
         mpco2 = xr.Dataset({recon:(['time','lat','lon'],ml_gridded[recon].sel(model=f"{model}")),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    if model=="ens":
        mpco2 = xr.Dataset({recon:(['time','lat','lon'],ml_gridded[recon]),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    if ((recon!="pCO2cc") & (model!="ens")):
        mpco2 = xr.Dataset({recon:(['time','lat','lon'],ml_gridded[recon]),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
  
    regridder = xe.Regridder(mpco2, lgrid, 'bilinear')
    regridder.periodic = 'true'
    
    if ((model=='ens')):
         pco2_new = regridder(ml_gridded[recon])
    if ((recon=="pCO2cc") & (model!='ens')):
        pco2_new = regridder(ml_gridded[recon].sel(model=f"{model}"))
    if ((recon!="pCO2cc") & (model!="ens")):
        pco2_new = regridder(ml_gridded[recon])
        
    ldeo_pco2 = ldeo_pco2.where(((ldeo_pco2<850) & (ldeo_pco2>150)))
    ldeo_pco2 = ldeo_pco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    pco2_new = pco2_new.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    

    # stack
    lpco2 = ldeo_pco2.stack(level=['time','lat','lon'])
    pco2_stack = pco2_new.stack(level=['time','lat','lon'])
    
    indx = (~np.isnan(lpco2.values) & ~np.isnan(pco2_stack.values))
    
    R = np.corrcoef(lpco2[indx],pco2_stack[indx])[0,1]
    LDEO_STD = np.std(lpco2[indx].values)
    STD = np.std(pco2_stack[indx].values)
    #RMSE
    RMSE = np.sqrt(np.square(lpco2[indx].values-pco2_stack[indx].values).sum()/indx.sum())
    
    return R, STD, RMSE, LDEO_STD

In [11]:
def GLODAP_stats(ml_gridded,model,recon,start_yr=1982,end_yr=2018):
    
    # Reconstructions #
    lon = ml_gridded.lon
    lat = ml_gridded.lat
    start_yr = max(start_yr,1985)
        
    start_yr = max(start_yr,1985)
    end_yr = min(end_yr,2018)
    
    mtime = ml_gridded.time
    
    # Load the data #
    glod = xr.open_dataset('/data/artemis/observations/GLODAP_v2/processed/GLODAPv2_spco2_1x1_198201-201812.nc')
    glod_pco2=glod.spco2_mean
    glod_lat=glod.lat
    glod_lon=glod.lon
    glod_time = glod.time
    
    # Deal with NaNs #
    if ((recon=="pCO2cc") & (model!='ens')):
        tmp = ml_gridded[recon].sel(model=f"{model}").where(ml_gridded[recon].sel(model=f"{model}") > 0)
    if model=="ens":
        tmp = ml_gridded[recon].where(ml_gridded[recon] > 0)
    if ((recon!="pCO2cc") & (model!="ens")):
        tmp = ml_gridded[recon].where(ml_gridded[recon] > 0)
    
    tmp = tmp.where(tmp < 850)
    
    # Regrid to GLODAP grid #
    mgrid = xr.Dataset({'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    ggrid = xr.Dataset({'lat':(['lat'],glod_lat.values),'lon':(['lon'],glod_lon.values)})
    mpco2 = xr.Dataset({recon:(['time','lat','lon'],tmp.values),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    regridder = xe.Regridder(mpco2, ggrid, 'bilinear')
    regridder.periodic = 'true'
    pco2_new = regridder(tmp)
    
    # Extract Time #
    glod_pco2 = glod_pco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    pco2_new = pco2_new.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    
    # Stack
    gpco2_stack = glod_pco2.stack(level=['time','lat','lon'])
    pco2_stack = pco2_new.stack(level=['time','lat','lon'])
    
    # Stats
    indx = ((~np.isnan(gpco2_stack.values)) & (~np.isnan(pco2_stack.values)))
    R = np.corrcoef(gpco2_stack[indx],pco2_stack[indx])[0,1]
    GLODAP_STD = np.std(gpco2_stack[indx].values)
    STD = np.std(pco2_stack[indx].values)
    #RMSE
    RMSE = np.sqrt(np.square(gpco2_stack[indx].values-pco2_stack[indx].values).sum()/indx.sum())
    
    return R, STD, RMSE, GLODAP_STD